In [13]:
import imageio as io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py as h5
from ipywidgets import interact,IntSlider
import os
from scipy.ndimage import measurements
from PIL import Image
from PIL.TiffTags import TAGS
from scipy.optimize import curve_fit
from scipy.stats import iqr
import nd2


## Here we load in the images and save them to png
##### this is because ilastik (to make smart masks) handles pngs. Download ilastik here https://www.ilastik.org/ and use a two color segmentation with the first color corresponding to the background, the second to the pillars, and use the batch_export module to create predictions for all your images

In [14]:
fnames_gel=[el for el in os.listdir('gel_avg_replicate') if el.endswith('.tif')]
if not os.path.exists('gel_repli_png/'):
    os.mkdir('gel_repli_png/')
for i in range(len(fnames_gel)):
    with Image.open('gel_avg_replicate/'+fnames_gel[i]) as img:
        meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.keys()}
        mum_per_pix=1/(meta_dict['XResolution'][0][0]/1e6)
        #you can add the resolution into the file_name if needed for later
        #processing of lengths
        #print(mum_per_pix)
        #plt.imshow(img)
        img=np.asarray(img)
        ma=np.min(img)+iqr(img,rng=(0,99.5))
        img=np.clip(img,0,ma)
        img=(img-np.min(img))/ma
        new_image_name='gel_repli_png/'+fnames_gel[i].replace('.tif','.png')
        if not os.path.exists(new_image_name):
            io.imsave(new_image_name,img)

In [15]:
fnames_chip=[el for el in os.listdir('chip_replicate') if el.endswith('.nd2')]
if not os.path.exists('chip_repli_png/'):
    os.mkdir('chip_repli_png/')
for i in range(len(fnames_chip)):
    img = nd2.imread('chip_replicate/'+fnames_chip[i])                          # read to numpy array
    f = nd2.ND2File('chip_replicate/'+fnames_chip[i])
    img=np.asarray(img)
    ma=np.min(img)+iqr(img,rng=(0,99.5))
    img=np.clip(img,0,ma)
    img=(img-np.min(img))/ma
    new_image_name='chip_repli_png/'+fnames_chip[i].replace('.nd2','.png')
    if not os.path.exists(new_image_name):
        io.imsave(new_image_name,img)

## Chip statistics
### Requires that you export the predictions from ilastik into h5 format

In [121]:
path='gel_repli_png/'
image_names=sorted([path+el for el in os.listdir(path) if el.endswith('.png')])
h5_names=sorted([path+el for el in os.listdir(path) if el.endswith('.h5')])
unique_names,indices=np.unique([el[:-6] for el in image_names],return_index=True)
indices=np.hstack((indices,len(image_names)))

res_folder='Results_'+path[:-5]+'/'
#os.mkdir(res_folder)

all_lengths=[]
for i in range(len(image_names)):

    mask=h5.File(h5_names[i],'r')['exported_data'][:,:,1]>0.5
    labeled,_=measurements.label(mask)

    from skimage.measure import regionprops
    regs=regionprops(labeled)
    regs=[r for r in regs if r.area>2]
    xy=[np.mean(el.coords,axis=0) for el in regs]

    y,x=np.asarray(xy)[:,:2].T
    from scipy.spatial import Delaunay
    points=np.vstack((x,y)).T
    tri=Delaunay(points)

    #clean up degenerate triangles from the samples edge (very flat triangles)
    triz=points[tri.simplices]
    angle= lambda x,y: np.abs(np.dot(y,x)/(np.sqrt(np.sum(x*x))*np.sqrt(np.sum(y*y))))
    get_angles = lambda x: np.median(np.array([angle(x[1]-x[2],x[1]-x[0]),angle(x[1]-x[2],x[2]-x[0]),angle(x[1]-x[0],x[2]-x[0])]))
    non_degenerate_trigs=np.asarray(list(map(get_angles,triz)))<0.7
    tri.simplices=tri.simplices[non_degenerate_trigs]

    length= lambda x: np.sqrt(np.sum(x*x))
    get_lens= lambda x: np.array([length(x[1]-x[2]),length(x[2]-x[0]),length(x[1]-x[0])])
    lengths=np.asarray(list(map(get_lens,points[tri.simplices]))).flatten()
    all_lengths.append(lengths*.31074033574)


In [128]:
for idx,l in enumerate(all_lengths):
    #low=np.min(l)+iqr(l,rng=(0,5))
    low=8
    #high=np.min(l)+iqr(l,rng=(0,99))
    high=18
    a,b=np.histogram(l,np.linspace(low,high,50))
    np.savetxt(res_folder+image_names[idx].split('/')[1].replace('.png','.txt'),l)

    plt.xlabel('Distance between pillars [um]',fontsize=15)
    plt.ylabel('Count',fontsize=15)
    from scipy.optimize import curve_fit


    b=(b[1:]+b[:-1])/2 # for len(x)==len(y)

    def gauss(x,mu,sigma,A):
        return A*np.exp(-(x-mu)**2/2/sigma**2)

    def bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2):
        return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

    expected=(11,0.5,6000,15.5,0.5,3000)
    params,cov=curve_fit(bimodal,b,a,expected)
    sigma=np.sqrt(np.diag(cov))
    _=plt.hist(l,np.linspace(low,high,50),rwidth=0.8)
    x=np.linspace(low,high,1000)
    plt.plot(x,bimodal(x,*params),color='red',lw=1,label='model')
    avg=np.abs(params[1]/params[0])
    plt.text(13,0.8*np.max(a),r'$\frac{\sigma}{\mu}=$'+str(np.round(100*avg,1))+'$\%$',fontsize=15)
    plt.title('Expanded gel')
    plt.xlim(low,high)
    plt.savefig(res_folder+image_names[idx].split('/')[1])
    plt.close()

In [129]:
path='chip_repli_png/'
image_names=sorted([path+el for el in os.listdir(path) if el.endswith('.png')])
h5_names=sorted([path+el for el in os.listdir(path) if el.endswith('.h5')])
unique_names,indices=np.unique([el[:-6] for el in image_names],return_index=True)
indices=np.hstack((indices,len(image_names)))

res_folder='Results_'+path[:-5]+'/'
#os.mkdir(res_folder)

all_lengths=[]
for i in range(len(image_names)):

    mask=h5.File(h5_names[i],'r')['exported_data'][:,:,1]>0.5
    labeled,_=measurements.label(mask)

    from skimage.measure import regionprops
    regs=regionprops(labeled)
    regs=[r for r in regs if r.area>10]
    xy=[np.mean(el.coords,axis=0) for el in regs]

    y,x=np.asarray(xy)[:,:2].T
    from scipy.spatial import Delaunay
    points=np.vstack((x,y)).T
    tri=Delaunay(points)

    #clean up degenerate triangles from the samples edge (very flat triangles)
    triz=points[tri.simplices]
    angle= lambda x,y: np.abs(np.dot(y,x)/(np.sqrt(np.sum(x*x))*np.sqrt(np.sum(y*y))))
    get_angles = lambda x: np.median(np.array([angle(x[1]-x[2],x[1]-x[0]),angle(x[1]-x[2],x[2]-x[0]),angle(x[1]-x[0],x[2]-x[0])]))
    non_degenerate_trigs=np.asarray(list(map(get_angles,triz)))<0.7
    tri.simplices=tri.simplices[non_degenerate_trigs]

    length= lambda x: np.sqrt(np.sum(x*x))
    get_lens= lambda x: np.array([length(x[1]-x[2]),length(x[2]-x[0]),length(x[1]-x[0])])
    lengths=np.asarray(list(map(get_lens,points[tri.simplices]))).flatten()
    all_lengths.append(lengths*.12787666)


In [130]:
for idx,l in enumerate(all_lengths):
    #low=np.min(l)+iqr(l,rng=(0,5))
    low=0
    #high=np.min(l)+iqr(l,rng=(0,99))
    high=6
    a,b=np.histogram(l,np.linspace(low,high,100))
    np.savetxt(res_folder+image_names[idx].split('/')[1].replace('.png','.txt'),l)

    plt.xlabel('Distance between pillars [um]',fontsize=15)
    plt.ylabel('Count',fontsize=15)
    from scipy.optimize import curve_fit


    b=(b[1:]+b[:-1])/2 # for len(x)==len(y)

    def gauss(x,mu,sigma,A):
        return A*np.exp(-(x-mu)**2/2/sigma**2)

    def bimodal(x,mu1,sigma1,A1,mu2,sigma2,A2):
        return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

    expected=(2.,0.1,np.max(a),3.4,0.1,np.max(a))
    params,cov=curve_fit(bimodal,b,a,expected)
    sigma=np.sqrt(np.diag(cov))
    _=plt.hist(l,np.linspace(low,high,50),rwidth=0.8)
    x=np.linspace(low,high,1000)
    plt.plot(x,bimodal(x,*params),color='red',lw=1,label='model')
    avg=np.abs(params[1]/params[0])
    plt.text(1,0.8*np.max(a),r'$\frac{\sigma}{\mu}=$'+str(np.round(100*avg,1))+'$\%$',fontsize=15)
    plt.title('Chip')
    plt.xlim(low,high)
    plt.savefig(res_folder+image_names[idx].split('/')[1],bbox_inches='tight',pad_inches=0.5)
    plt.close()